In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import random
import json
import os
import time
import tensorflow as tf
from tensorflow.contrib import rnn


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
#from sklearn.model_selection import train_test_split
import tensorflow.contrib.legacy_seq2seq as seq2seq
from utilities import show_graph
#from util import inv_sigmoid, linear_decay, dec_print_train, dec_print_val, dec_print_test

import unicodedata
import re
import numpy as np
import os
import io
import time
import collections
import json
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
random.seed(0)
np.random.seed(0)
tf.set_random_seed(0)

n_inputs        = 4096
n_hidden        = 600
val_batch_size  = 100 #100
n_frames        = 80
max_caption_len = 50
forget_bias_red = 1.0
forget_bias_gre = 1.0
dropout_prob    = 0.5
n_attention     = n_hidden

special_tokens  = {'<PAD>': 0, '<BOS>': 1, '<EOS>': 2, '<UNK>': 3}
phases = {'train': 0, 'val': 1, 'test': 2}

In [3]:
#The following function was taken from: https://github.com/AdrianHsu/S2VT-seq2seq-video-captioning-attention

class S2VT:
    def __init__(self, vocab_num = 0,lr = 1e-4):

        self.vocab_num = vocab_num
        self.learning_rate = lr

     
    def build_model(self, feat, captions=None, cap_len=None, sampling=None, phase=0):

        weights = {
            'W_feat': tf.Variable( tf.random_uniform([n_inputs, n_hidden], -0.1, 0.1), name='W_feat'), 
            'W_dec': tf.Variable(tf.random_uniform([n_hidden, self.vocab_num], -0.1, 0.1), name='W_dec')
        }
        biases = {
            'b_feat':  tf.Variable( tf.zeros([n_hidden]), name='b_feat'),
            'b_dec': tf.Variable(tf.zeros([self.vocab_num]), name='b_dec')
        }   
        embeddings = {
         'emb': tf.Variable(tf.random_uniform([self.vocab_num, n_hidden], -0.1, 0.1), name='emb')
        }

        batch_size = tf.shape(feat)[0]

        # cap_len: (250, 1) -> (250, 50)
        cap_mask = tf.sequence_mask(cap_len, max_caption_len, dtype=tf.float32)
     
        if phase == phases['train']: #  add noise
            noise = tf.random_uniform(tf.shape(feat), -0.1, 0.1, dtype=tf.float32)
            feat = feat + noise

        if phase == phases['train']:
            feat = tf.nn.dropout(feat, dropout_prob)

        feat = tf.reshape(feat, [-1, n_inputs])
        image_emb = tf.matmul(feat, weights['W_feat']) + biases['b_feat']
        image_emb = tf.reshape(image_emb, [-1, n_frames, n_hidden])
        image_emb = tf.transpose(image_emb, perm=[1, 0, 2])
        
        with tf.variable_scope('LSTM1'):
            lstm_red = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=forget_bias_red, state_is_tuple=True)
            if phase == phases['train']:
                lstm_red = tf.contrib.rnn.DropoutWrapper(lstm_red, output_keep_prob=dropout_prob)    
        with tf.variable_scope('LSTM2'):
            lstm_gre = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=forget_bias_gre, state_is_tuple=True)
            if phase == phases['train']:
                lstm_gre = tf.contrib.rnn.DropoutWrapper(lstm_gre, output_keep_prob=dropout_prob)    

        state_red = lstm_red.zero_state(batch_size, dtype=tf.float32)
        state_gre = lstm_gre.zero_state(batch_size, dtype=tf.float32)

        padding = tf.zeros([batch_size, n_hidden])

#         h_src = []
        for i in range(0, n_frames):
            with tf.variable_scope("LSTM1"):
                output_red, state_red = lstm_red(image_emb[i,:,:], state_red)
            
            with tf.variable_scope("LSTM2"):
                output_gre, state_gre = lstm_gre(tf.concat([padding, output_red], axis=1), state_gre)
#                 h_src.append(output_gre) # even though padding is augmented, output_gre/state_gre's shape not change

#         h_src = tf.stack(h_src, axis = 0)

        bos = tf.ones([batch_size, n_hidden])
        padding_in = tf.zeros([batch_size, n_hidden])

        logits = []
        max_prob_index = None

        

        cross_ent_list = []
        for i in range(0, max_caption_len):

            with tf.variable_scope("LSTM1"):
                output_red, state_red = lstm_red(padding_in, state_red)

            if i == 0:
                with tf.variable_scope("LSTM2"):
                    con = tf.concat([bos, output_red], axis=1)
                    output_gre, state_gre = lstm_gre(con, state_gre)
            else:
                if phase == phases['train']:
                    if sampling[i] == True:
                        feed_in = captions[:, i - 1]
                    else:
                        feed_in = tf.argmax(logit_words, 1)
                else:
                    feed_in = tf.argmax(logit_words, 1)
                with tf.device("/cpu:0"):
                    embed_result = tf.nn.embedding_lookup(embeddings['emb'], feed_in)
                with tf.variable_scope("LSTM2"):
                    con = tf.concat([embed_result, output_red], axis=1)
                    output_gre, state_gre = lstm_gre(con, state_gre)

            logit_words = tf.matmul(output_gre, weights['W_dec']) + biases['b_dec']
            logits.append(logit_words)

#             if phase != phases['test']:
            labels = captions[:, i]
            one_hot_labels = tf.one_hot(labels, self.vocab_num, on_value = 1, off_value = None, axis = 1) 
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logit_words, labels=one_hot_labels)
            cross_entropy = cross_entropy * cap_mask[:, i]
            cross_ent_list.append(cross_entropy)
        
        loss = 0.0
#         if phase != phases['test']:
        cross_entropy_tensor = tf.stack(cross_ent_list, 1)
        loss = tf.reduce_sum(cross_entropy_tensor, axis=1)
        loss = tf.divide(loss, tf.cast(cap_len, tf.float32))
        loss = tf.reduce_mean(loss, axis=0)

        logits = tf.stack(logits, axis = 0)
        logits = tf.reshape(logits, (max_caption_len, batch_size, self.vocab_num))
        logits = tf.transpose(logits, [1, 0, 2])
        
        summary = None
        if phase == phases['train']:
            summary = tf.summary.scalar('training_loss', loss)
        elif phase == phases['val']:
            summary = tf.summary.scalar('validation_loss', loss)
            

        return logits, loss, summary

    def inference(self, logits):
        
        #print('using greedy search...')
        dec_pred = tf.argmax(logits, 2)
        return dec_pred

    def optimize(self, loss_op):

        params = tf.trainable_variables()
        optimizer = tf.train.AdamOptimizer(self.learning_rate)#.minimize(loss_op)
        gradients, variables = zip(*optimizer.compute_gradients(loss_op))
        gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
        train_op = optimizer.apply_gradients(zip(gradients, params))

        return train_op

In [4]:
def tokenize(line,token='word'):
    if token == 'word':
        return [line.split(' ')]
    elif token == 'char':
        return [list(line)]
    else:
        print('ERROR: unknown token type '+token)

In [5]:
def count_tokens(tokanized_sentences):
    # Flatten a list of token lists into a list of tokens
    tokens = [tk for line in tokanized_sentences for tk in line]
    return collections.Counter(tokens)

In [6]:
def parse_data_into_lists(filename,batch_size,feat_filepath, index2token, tokens):
    
    with open(filename, 'r') as f:
        datastore = json.load(f)
           
    #sentence_set = extract_sentences(filename)
    
    mult_vids = []
    all_sents = []
    all_enc_sents = []
    all_cap_len = []
    all_ids = []
    
    for data in datastore:
        
        sentences = data["caption"]
        sentences = [word.lower() for word in sentences] #Normalize the case
        table = str.maketrans('', '', string.punctuation) #Normalize the punctuation
        sentences = [word.translate(table) for word in sentences]
        
        num_sent = len(sentences)
        
        all_sents.extend(sentences)
        
        enc_sents = []
        
        for sentence in sentences:
            
            #print(sentence)
            tokenized_sentence, encoded_sentence, cap_len = num_encode(sentence,index2token,tokens)
            #print(tokenized_sentence)
            #print(encoded_sentence)
            #print(cap_len)
            #print(encoded_sentence)
            encoded_sentence = list(encoded_sentence)

            enc_sents.append(encoded_sentence)
            all_cap_len.append(cap_len)
            
        all_enc_sents.extend(enc_sents)


#         print(all_sents[0])
#         print(all_enc_sents[0])
#         print(np.shape(all_enc_sents))
#         print(np.shape(all_sents))
#         print(np.shape(all_cap_len))

        #print(len(all_enc_sents))

        #sentence_set[i] = sentences 
        #### Extracting all feature vectors per video
        
        video_id = data["id"]
        features = np.load(feat_filepath.format(video_id))
        
        for n in range(0,num_sent):
            mult_vids.append(features)
            all_ids.append(video_id)
        
        print("id: " + str(video_id) + " processed")

            
    return mult_vids, all_sents, all_enc_sents, all_cap_len, all_ids

In [7]:

def parse_data_into_batches(filename,batch_size,feat_filepath, index2token, tokens, mult_vids, all_sents, all_enc_sents, all_cap_len, all_ids):
    #print(all_enc_sents[0:10])

    with open(filename, 'r') as f:
        datastore = json.load(f)
           
    vid_batch = {}
    sent_batch = {}
    enc_sent_batch = {}
    cap_len_batch = {}
    id_batch = {}
    
    #sentence_set = extract_sentences(filename)
    
#     mult_vids = []
#     all_sents = []
#     all_enc_sents = []
#     all_cap_len = []
    
#     for data in datastore:
        
#         sentences = data["caption"]
#         sentences = [word.lower() for word in sentences] #Normalize the case
#         table = str.maketrans('', '', string.punctuation) #Normalize the punctuation
#         sentences = [word.translate(table) for word in sentences]
        
#         num_sent = len(sentences)
        
#         all_sents.extend(sentences)
        
#         for sentence in sentences:
#             tokenized_sentence, encoded_sentence, cap_len = num_encode(sentence,index2token,tokens)
            
#             #print(encoded_sentence)
#             all_enc_sents.append(encoded_sentence)
#             all_cap_len.append(cap_len)

#         #print(all_enc_sents[0])
#         #print(len(all_enc_sents))

#         #sentence_set[i] = sentences 
#         #### Extracting all feature vectors per video
        
#         video_id = data["id"]
#         features = np.load(feat_filepath.format(video_id))
        
#         for n in range(0,num_sent):
#             mult_vids.append(features)
            
#         #print("id: " + str(data["id"]) + " processed")


    random.Random(30).shuffle(mult_vids)
    random.Random(30).shuffle(all_sents)
    random.Random(30).shuffle(all_enc_sents)
    random.Random(30).shuffle(all_cap_len)
    random.Random(30).shuffle(all_ids)
    
#     print(all_enc_sents[0:5])
#     print(all_sents[0:5])
    
    batches = len(mult_vids)/batch_size
    batches = int(batches)
        
    i = 0
    j = 0
    
    for n in range(0,batches):
        if j not in vid_batch:
            vid_batch[j] = []
            sent_batch[j] = []
            enc_sent_batch[j]=[]
            cap_len_batch[j] = []

            
        vid_batch[j] = mult_vids[i:i+batch_size]
        sent_batch[j] = all_sents[i:i+batch_size]
        enc_sent_batch[j] = all_enc_sents[i:i+batch_size]
        cap_len_batch[j] = all_cap_len[i:i+batch_size]
        id_batch[j] = all_ids[i:i+batch_size]

        print('parsed batch %d' %j)
        i = i+batch_size
        j = j+1
    
#     if j not in vid_batch:
#         vid_batch[j] = []

#     vid_batch[j].append(features)
                
#     i = i+1

#     if i%batch_size == 0:
#         j = j+1 
    #print(sent_batch[0])
    #print(enc_sent_batch[0])
            
    return vid_batch, sent_batch, enc_sent_batch, cap_len_batch, batches, id_batch

In [8]:
def extract_sentences(filename):
    
    sentence_set = {}
    
    with open(filename, 'r') as f:
        datastore = json.load(f)
        
    i = 0
    for data in datastore:
        
        #### Extracting only a single sentence per video into a standalone dict

        sentences = data["caption"]
        sentences = [word.lower() for word in sentences] #Normalize the case
        table = str.maketrans('', '', string.punctuation) #Normalize the punctuation
        sentences = [word.translate(table) for word in sentences]

        sentence_set[i] = sentences #0 for only the first sentence\
        
        i = i+1
        
    return sentence_set

In [9]:
# Mapping string tokens to numertical indices.
def listVocab(sentence_set):
    
    PAD_token = 0
    BOS_token = 1
    EOS_token = 2
    UNK_token = 3
    
    all_tokens = []
    word_count = {}
    token2index = {"<PAD>": 0,"<BOS>":1,"<EOS>":2,"<UNK>":3}
    index2token = {PAD_token: "<PAD>", BOS_token: "<BOS>", EOS_token: "<EOS>", UNK_token: "<UNK>"}
    
    for set_i in sentence_set:
        sentence_set_i = sentence_set[set_i]
        for line in sentence_set_i:
#             line = sentence_set[n]
            tokenized_captions = tokenize(line) #Seperate the words
            all_tokens += tokenized_captions
    
    counter = count_tokens(all_tokens) #Count the word repeatitions in each set
    
    counter_dict = counter.items()
    counter_sort = sorted(counter_dict, key=lambda x:x[1],reverse=True) #sort by frequency of occurance 
    #print(counter_sort)

    i = len(index2token)
    values = [0,1,2,3]
    tokens = ["<PAD>","<BOS>","<EOS>","<UNK>"]
    
    for token, freq in counter_sort:
        word_count[token] = freq
        index2token[i] = token
        token2index[token] = i
        values += [i]
        tokens += [token]
        i+=1
        
    word_count['<PAD>'] = i
    word_count['<BOS>'] = i
    word_count['<EOS>'] = i
    word_count['<UNK>'] = i
    
    bias_init_vector = np.array([1.0 * word_count[ index2token[i] ] for i in index2token])
    bias_init_vector /= np.sum(bias_init_vector) # normalize to frequencies
    bias_init_vector = np.log(bias_init_vector)
    bias_init_vector -= np.max(bias_init_vector) # shift to nice numeric range
    
    return [word_count, tokens, values, token2index, index2token, len(index2token),bias_init_vector]

In [10]:
def flattenList(nestedList,output): 
    for i in nestedList: 
        if type(i) == list: 
            flattenList(i,output) 
        else: 
            output.append(i) 
            
    return output

def num_encode(test_sentence,index2token,tokens,tokenized_sentence=[],num_encoded_sentence=[]):
    
    tokenized_sentence.clear()
    num_encoded_sentence.clear()
    
    tokenized_sentence = ["<BOS>"] + tokenize(test_sentence) + ["<EOS>"]
    #print(tokenized_sentence)
    output=[]
    tokenized_sentence = flattenList(tokenized_sentence,output)
    
    cap_len = len(tokenized_sentence)
    
    while len(tokenized_sentence) < MAX_WORDS:
        tokenized_sentence.append("<PAD>")    
    
    #print(len(tokenized_sentence))
    
    for ind, token in enumerate(tokenized_sentence):
        if token in tokens:
            for i in range(0,len(index2token)):
                if token == index2token[i]: 
                    num_encoded_sentence.append(i) 
                    
            #print("token exists")
        else:
            num_encoded_sentence.append(3)
            tokenized_sentence[ind] = tokens[3]
            #print("token unknown")
            
            
                
    #print(len(num_encoded_sentence))

        
    return tokenized_sentence, num_encoded_sentence, cap_len

In [12]:
filename_train = 'MLDS_hw2_1_data/training_label.json'
feat_filepath_train = "MLDS_hw2_1_data/training_data/feat/{}.npy"
filename_test = 'MLDS_hw2_1_data/testing_label.json'
feat_filepath_test = "MLDS_hw2_1_data/testing_data/feat/{}.npy"

batch_size_test = 100


ckpt_path = 'saved_model/trained_model.ckpt'

MAX_WORDS = max_caption_len #max number of words in a caption
n_features = n_inputs
no_of_frames = n_frames
sizeof_sentence= MAX_WORDS
n_hidden = 1000

#### Extract TRAINING CAPTIONS #####

sentence_set = extract_sentences(filename_train)

_, tokens, values, token2index, index2token, n_words, bias_init_vector = listVocab(sentence_set)
print("There are %d unique words in the captions dataset" % n_words)

# #### PARSE TESTING DATA #####

# Extracting captions for each video
#sentence_set = extract_sentences(filename_test)

mult_vids_test, all_sents_test, all_enc_sents_test, all_cap_len_test, all_ids_test = parse_data_into_lists(filename_test,\
                                                                                  batch_size_test,\
                                                                                  feat_filepath_test,\
                                                                                  index2token,\
                                                                                  tokens)                                                 


There are 6061 unique words in the captions dataset
id: ScdUht-pM6s_53_63.avi processed
id: wkgGxsuNVSg_34_41.avi processed
id: BtQtRGI0F2Q_15_20.avi processed
id: k06Ge9ANKM8_5_16.avi processed
id: sZf3VDsdDPM_107_114.avi processed
id: shPymuahrsc_5_12.avi processed
id: XOAgUVVwKEA_8_20.avi processed
id: ufFT2BWh3BQ_0_8.avi processed
id: 5YJaS2Eswg0_22_26.avi processed
id: lw7pTwpx0K0_38_48.avi processed
id: UbmZAe5u5FI_132_141.avi processed
id: xCFCXzDUGjY_5_9.avi processed
id: He7Ge7Sogrk_47_70.avi processed
id: tJHUH9tpqPg_113_118.avi processed
id: n016q1w8Q30_2_11.avi processed
id: RjpbFlOHFps_8_25.avi processed
id: 6JnGBs88sL0_4_10.avi processed
id: EpMuCrbxE8A_107_115.avi processed
id: HAjwXjwN9-A_16_24.avi processed
id: 4xVGpDmA4lE_23_33.avi processed
id: k5OKBX2e7xA_19_32.avi processed
id: Jag7oTemldY_12_25.avi processed
id: 8MVo7fje_oE_125_130.avi processed
id: bqMmyY1ImkI_0_14.avi processed
id: jTnrm338_KY_34_42.avi processed
id: UdcObAQ5OOM_15_30.avi processed
id: 4PcL6-mjR

NameError: name 'all_ids_testtest' is not defined

In [13]:
vid_batch_test, sent_batch_test, intencode_batch_test, cap_len_batch_test, n_batches_test, id_batch_test = parse_data_into_batches(filename_test,\
                                                                                           batch_size_test,\
                                                                                           feat_filepath_test,\
                                                                                           index2token, \
                                                                                           tokens, \
                                                                                           mult_vids_test, \
                                                                                           all_sents_test, \
                                                                                           all_enc_sents_test, \
                                                                                           all_cap_len_test,\
                                                                                           all_ids_test)     

[[1, 4, 1594, 1093, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 4, 7, 5, 163, 14, 22, 147, 9, 6, 121, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 4, 90, 5, 42, 68, 26, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 4, 5987, 709, 48, 4915, 6, 2037, 768, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 4, 28, 5, 43, 422, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
['a missile explodes', 'a man is jumping with his hands on the ground', 'a monkey is walking through water', 'a bomb went off beyond the palm trees', 'a girl is eating noodles']
parsed batch 0
parsed bat

In [14]:
def schedule_sampling(sampling_prob, cap_len_batch):

        sampling = np.ones(max_caption_len, dtype = bool)
        for l in range(max_caption_len):
            if np.random.uniform(0,1,1) < sampling_prob:
                sampling[l] = True
            else:
                sampling[l] = False
         
        sampling[0] = True
        return sampling

In [15]:
def inv_sigmoid(num_epo):

    # 0.88 to 0.12 (-2.0 to 2.0)
    x = np.arange(-2.0, 2.0, (4.0/num_epo))
    y = 1/(1 + np.e**x)
    #y = np.ones(num_epo)
    #print(y)
    return y

In [16]:
def pred_print(pred, cap_len, label, idx2word, batch_size, id_batch):
    
    print_this = np.random.randint(batch_size,size=(1, 10))
    seq=[]
    for i in range(0,batch_size):
        eos_pred = max_caption_len - 1
        eos = cap_len[i] - 1
        for j in range(0, max_caption_len):
                if pred[i][j] == special_tokens['<EOS>']:
                    eos_pred = j
                    break
        myid = id_batch[i]
        pre = list( map (lambda x: idx2word[x] , pred[i][0:eos_pred])  )
        lab = list( map (lambda x: idx2word[x] , label[i][0:eos])  )
        
        pre_no_eos = list( map (lambda x: idx2word[x] , pred[i][0:(eos_pred)])  )
        sen = ' '.join([w for w in pre_no_eos])
        seq.append(sen)
        if i in print_this:      
            print('\nid: ' + str(myid) + '\nanswer: ' + str(lab) + '\nprediction: ' + str(pre))
            
    return seq

In [18]:
tf.reset_default_graph()

from tqdm import tqdm
val_graph = tf.Graph()

gpu_config = tf.ConfigProto()
gpu_config.gpu_options.allow_growth = True

vocab_num = n_words
num_epochs = 10
num_display_steps = 15
num_saver_epochs = 3
output_filename = 'output.txt'
learning_rate = 0.0001


with val_graph.as_default():
    feat_val = tf.placeholder(tf.float32, [None, n_frames, n_inputs], name='video_features')
    captions_val = tf.placeholder(tf.int32, [None, max_caption_len], name='captions')
    cap_len_val = tf.placeholder(tf.int32, [None], name='cap_len')

    model_val = S2VT(vocab_num=vocab_num, lr=learning_rate)
    logits_val, loss_op_val, summary_val = model_val.build_model(feat_val, 
                captions_val, cap_len_val, phase=phases['val'])
    dec_pred_val = model_val.inference(logits_val)

    val_saver = tf.train.Saver(max_to_keep=3)
    
val_sess = tf.Session(graph=val_graph, config=gpu_config)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [20]:
ckpt_path = 'saved_model/trained_model.ckpt-959'

val_saver.restore(val_sess, ckpt_path)
val_sess.run(init)


epo_loss_val = 0
txt = open(output_filename, 'w')

total_loss_val = 0

for j in range(0,n_batches_test):
    data_batch_val = np.array(vid_batch_test[j])
    label_batch_val = np.array(intencode_batch_test[j])
    id_batch_val = id_batch_test[j]
    caption_lens_batch_val = np.array(cap_len_batch_test[j])

    loss_val, p_val, summ = val_sess.run([loss_op_val, dec_pred_val, summary_val], 
                                feed_dict={feat_val: data_batch_val,
                                           captions_val: label_batch_val,
                                           cap_len_val: caption_lens_batch_val})
    
    print(loss_val)

    seq_val = pred_print(p_val, caption_lens_batch_val, label_batch_val, index2token, batch_size_test, id_batch_val)
    total_loss_val += loss_val

    for k in range(0, batch_size_test):
            txt.write(id_batch_val[k] + "," + seq_val[k] + "\n")

print('\nSave file: ' + output_filename)
txt.close()
from subprocess import call

call(['python3', 'MLDS_hw2_1_data/bleu_eval.py', output_filename])

print("Validation: " + str((j+1) * batch_size_test) + "/" + \
        str(n_batches_test) + ", done..." \
        + "Total Loss: " + "{:.4f}".format(total_loss_val))

INFO:tensorflow:Restoring parameters from saved_model/trained_model.ckpt-959


InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Assign requires shapes of both tensors to match. lhs shape= [2000,4000] rhs shape= [1200,2400]
	 [[node save/Assign_1 (defined at /home/adhitir/.conda/envs/tf_class/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'save/Assign_1':
  File "/home/adhitir/.conda/envs/tf_class/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/adhitir/.conda/envs/tf_class/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/adhitir/.conda/envs/tf_class/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/adhitir/.conda/envs/tf_class/lib/python3.7/site-packag

In [ ]:
seq = pred_print(p, caption_lens_batch, label_batch, index2token, batch_size, id_batch_train)


In [ ]:
np.random.randint(batch_size,10)


In [ ]:
# model = Video_Caption_Generator(dim_image=n_features, 
#                                 n_words = n_words, 
#                                 dim_hidden = n_hidden, 
#                                 batch_size=batch_size, 
#                                 n_lstm_steps=80,
#                                 n_video_lstm_step=80,
#                                 n_caption_lstm_step=80,
#                                 bias_init_vector=bias_init_vector)

In [ ]:
# tf_loss, tf_video, tf_video_mask, tf_caption, tf_caption_mask, tf_probs = model.build_model()


In [ ]:
# n_words = n_words

# with tf.Graph().as_default() as graph:
    

#     weights_enc = tf.Variable(tf.random_uniform([n_features, n_hidden],-0.1,0.1),name="weights_enc")
#     bias_enc = tf.Variable(tf.zeros([n_hidden]),name="bias_enc")

#     weights_dec = tf.Variable(tf.random_uniform([n_hidden, n_words],-0.1,0.1),name="weights_dec")
#     bias_dec = tf.Variable(tf.zeros([n_words]),name="bias_dec")


#     x_video = tf.placeholder(tf.float32, (None, no_of_frames, n_features),'video_features') #inputs

#     batch_size = tf.shape(x_video)[0]
    
#     x_video_drop = tf.nn.dropout(x_video, 0.5)
    
#     x_video_flat = tf.reshape(x_video_drop,[-1,n_features])

#     y_label = tf.placeholder(tf.int32,(None, sizeof_sentence),'captions') #outputs


#     #sampling = tf.placeholder(tf.bool, [sizeof_sentence], name='sampling')
#     padding = tf.zeros([batch_size, n_hidden])

#     loss = 0.0

#     ########## DATA ###########
#     # Example: For i = 0
#     #batch_x = np.array(vid_batch[0])
#     #batch_y = np.array(intencode_batch[0])
#     ###########################

#     input_embedding = tf.matmul(x_video_flat,weights_enc) + bias_enc
#     input_embedding = tf.reshape(input_embedding,[-1, no_of_frames,n_hidden])
#     input_embed = tf.transpose(input_embedding, perm=[1, 0, 2])

#     with tf.device("/cpu:0"):
#         output_embedding = tf.Variable(tf.random_uniform((n_words, n_hidden),-0.1,0.1), name='dec_embedding')
#     # output_embed = tf.nn.embedding_lookup(output_embedding,y_label)
    
#     ## ENCODING #################################
    
#     with tf.variable_scope("LSTM1"):
#         lstm1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden,state_is_tuple=True)
#         lstm1 = tf.contrib.rnn.DropoutWrapper(lstm1, output_keep_prob=0.5)    

#     with tf.variable_scope("LSTM2"):
#         lstm2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, state_is_tuple=True)
#         lstm2 = tf.contrib.rnn.DropoutWrapper(lstm2, output_keep_prob=0.5)    


#     state1 = lstm1.zero_state(batch_size, dtype=tf.float32)
#     state2 = lstm2.zero_state(batch_size, dtype=tf.float32)
    
#     for i in range(0, no_of_frames):
        
#         if i > 0:
#                 tf.get_variable_scope().reuse_variables()
                
#         with tf.variable_scope("LSTM1"):
#             output1, state1 = lstm1(input_embed[i,:,:], state1)

#         with tf.variable_scope("LSTM2"):
#             output2, state2 = lstm2(tf.concat([padding, output1], axis=1), state2)
    
#     ## DECODING ##################################
    
#     bos = tf.ones([batch_size, n_hidden])
#     padding_in = tf.zeros([batch_size, n_hidden])

#     logits = []
#     cross_ent_list=[]
#     max_prob_index = None


#     for i in range(0, MAX_WORDS):
        
#         tf.get_variable_scope().reuse_variables()

        
#         with tf.variable_scope("LSTM1"):
#             output1, state1 = lstm1(padding_in, state1)
            
#         if i == 0:
            
#             with tf.variable_scope("LSTM2"):
#                 con = tf.concat([bos, output1], axis=1)
#                 output2, state2 = lstm2(con, state2)
                
#         else:
            
#             with tf.device("/cpu:0"):
            
#                 feed_in = y_label[:,i]
#                 #feed_in = tf.argmax()
#                 output_embed = tf.nn.embedding_lookup(output_embedding,feed_in)
                
#             with tf.variable_scope("LSTM2"):
#                 con = tf.concat([output_embed, output1], axis=1)
#                 output2, state2 = lstm2(con, state2)

#         logit_words = tf.matmul(output2, weights_dec) + bias_dec
#         logits.append(logit_words)

#         word_i = y_label[:,i]

#         one_hot_labels = tf.one_hot(word_i, n_words, on_value = 1, off_value = None, axis = 1) 
#         cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logit_words, labels=one_hot_labels)
#         cross_ent_list.append(cross_entropy)
        
        
#         #current_loss = tf.reduce_sum(cross_entropy)/batch_size
#         #loss = loss + current_loss

#     cross_entropy_tensor = tf.stack(cross_ent_list, 1)
#     loss = tf.reduce_sum(cross_entropy_tensor, axis=1)
#     loss = tf.divide(loss, tf.cast(tf.Variable(sizeof_sentence), tf.float32))

#     loss = tf.reduce_mean(loss, axis=0)
    
#     summary = tf.summary.scalar('training_loss', loss)

#     params = tf.trainable_variables()
#     #optimizer = tf.train.AdamOptimizer(learning_rate)#.minimize(loss_op)
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#     train_op = optimizer.minimize(loss)

#     #train_step = optimizer.minimize(loss)
    
# #     gradients, variables = zip(*optimizer.compute_gradients(loss))
# #     gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
# #     train_op = optimizer.apply_gradients(zip(gradients, params))
    
# #     logits = tf.stack(logits, axis = 0)
# #     logits = tf.reshape(logits, (sizeof_sentence, batch_size, n_words))
# #     logits = tf.transpose(logits, [1, 0, 2])
# #     preds = tf.argmax(logits,2)
# #     correct_pred = tf.equal(tf.argmax(preds,1), tf.argmax(y_label,1))
# #     accuracy = tf.reduce_mean(correct_pred)

#     logits = tf.stack(logits,axis=0)
#     logits = tf.transpose(logits, [1, 0, 2])
#     output_preds = tf.argmax(logits,2)
    
#     #correct_pred = tf.equal(tf.argmax(output_preds, 1), tf.argmax(y_label, 1))
#     #accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
#     saver = tf.train.Saver(max_to_keep=3)


In [ ]:
# loss

In [ ]:
# run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

# gpu_config = tf.ConfigProto()

# with tf.Session(graph=graph,config=gpu_config) as sess:

#     loss_list_train = []
#     loss_list_test = []
#     preds_dict = {}

#     sess.run(tf.global_variables_initializer())
#     epochs = 10

#     #training
#     n=0

#     for epoch in range(epochs):

#         for i in range(n_batches):

#             batch_x = np.array(vid_batch[i])
#             #batch_x = np.reshape(batch_x,[-1,n_features])
#             batch_y = np.array(intencode_batch[i])

#             _, batch_loss, preds = sess.run([train_op, loss, logits], feed_dict = {x_video: batch_x, y_label: batch_y})        

#             loss_list_train.append(batch_loss)
#             print("train: %f " % (batch_loss))

            
#             n = n+1

       
#     #testing
    
#         saver.save(sess,ckpt_path, global_step=n)
#         print('Model saved at ' + ckpt_path)
        
    
#     for i in range(n_batches_test):

#         batch_x_test = np.array(vid_batch_test[i])
#         #batch_x = np.reshape(batch_x,[-1,n_features])
#         batch_y_test = np.array(intencode_batch_test[i])

#         acc = sess.run(accuracy, feed_dict = {x_video: batch_x_test, y_label: batch_y_test})        
#         print("accuracy %f" % acc)

# #         loss_list_test.append(batch_loss)
# #         print("test:", batch_loss)
    
# #         preds_dict[i] = batch_preds 

In [ ]:
# with tf.variable_scope("encoding") as encoding_scope:
#     lstm_enc = tf.contrib.rnn.BasicLSTMCell(n_hidden)
#     _, last_state = tf.nn.dynamic_rnn(lstm_enc, inputs=input_embed, dtype=tf.float32)

In [ ]:
# with tf.variable_scope("decoding") as decoding_scope:
#     # TODO: create the decoder LSTMs, this is very similar to the above
#     # you will need to set initial_state=last_state from the encoder
#     lstm_dec = tf.contrib.rnn.BasicLSTMCell(n_hidden)
#     dec_outputs, _ = tf.nn.dynamic_rnn(lstm_dec,inputs=output_embed, dtype=tf.float32)

In [ ]:
# #connect outputs to 
# logits = tf.contrib.layers.fully_connected(dec_outputs, num_outputs=len(index2token), activation_fn=None) 

# with tf.name_scope("optimization"):
#     # Loss function
#     loss = tf.contrib.seq2seq.sequence_loss(logits, targets, tf.ones([batch_size, sizeof_sentence]))
#     # Optimizer
#     optimizer = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

In [ ]:
# output_dec.get_shape().as_list()

In [ ]:
# state_dec[0].get_shape().as_list()

In [ ]:
# x_video.get_shape().as_list()

In [ ]:
# from utilities import show_graph
# show_graph(tf.get_default_graph().as_graph_def())

In [ ]:
# def RNN(x, weights1, biases1):
    
#     x = tf.unstack(x,no_of_frames,1)
    
#     lstm_encoder = tf.keras.layers.LSTM(n_hidden, return_state=True) #reuse=tf.AUTO_REUSE)
#     output_encoder,state_h,state_c = lstm_encoder(x) #,dtype=tf.float32)
#     encoder_states = [state_h,state_c]
    
#     decoder
    
#     return tf.matmul(output1[-1],weights1) + bias1

In [ ]:
# np.shape(vid_batch[1])

In [ ]:
# logits = RNN(x_video,weights1,bias1)
# prediction = tf.nn.softmax(logits)


# loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_label))


# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# train_op = optimizer.minimize(loss_op)

# # Evaluate model (with test logits, for dropout to be disabled)
# correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_label, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# batch_y = np.array(intencode_batch[1])
# np.shape(batch_y)

In [ ]:
# batch_x = np.array(vid_batch[0])
# print(np.shape(batch_x))
# batch_x = np.reshape(batch_x,[-1,n_features])
# np.shape(batch_x)

In [ ]:
# preds_dict[0]
# def predicted_sentence(preds_dict):
    

In [ ]:
# batch_x = np.array(vid_batch[0])
# batch_y = np.array(intencode_batch[0])

# with tf.Session() as sess:
#     sess.run(init)

#     sess.run(train_op, feed_dict={x_video: batch_x, y_label: batch_y})

In [ ]:


# image_emb = tf.nn.xw_plus_b(x_video, weights1, bias1) 
# #image_emb = tf.reshape(image_emb, [batch_size, no_of_frames, n_hidden])

# #lstm2 = tf.keras.layers.LSTMCell(n_hidden)

# padding = tf.zeros([batch_size, n_hidden])


# #Only read the frames


        

            
                
# logit_words = tf.nn.xw_plus_b(output2, weights2, bias2)
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logit_words,onehot_encoded)

# loss = tf.reduce_sum(cross_entropy)

In [ ]:
# with tf.Session() as sess:
#     with sess.as_default():
#         print(tf.nn.embedding_lookup(onehot_encoded,[1]).eval())

In [ ]:
# inputs